# 추천 시스템
***
## 유형
### 1. **콘텐츠 기반 필터링**
- User가 특정 Item을 선호하는 경우 해당 Item과 가장 유사한 Item을 추천하는 방식
- 영화, 음악 등의 컨텐츠에 주로 사용될 듯

### 2. **협업 필터링**
- User별로 Item에 대해 평가한 정보 혹은 이용 이력과 같은 사용자 행동 양식을 기반으로 추천을 수행하는 방식
- User가 아직 평가하지 않은 Item에 대해 평가한다.

#### 2-1. **최근접 이웃 협업 필터링**
- 메모리 협업 필터링이라고도 하며 User 기반과 Item 기반으로 다시 나뉜다.
- User 기반 : 특정 User와 유사한 다른 User들을 TOP-N으로 선정해 TOP-N User들이 좋아하는 Item을 추천하는 방식. 즉, 특정 User와 다른 User들 간의 유사도를 기준으로 추천.
- Item 기반 : User들이 해당 Item을 평가한 정보를 기반으로 추천. Item간의 유사성 활용이 아님. 
- User Based 보다 Item Based 추천이 성능이 더 좋다. 

#### 2-2. **잠재 요인 협업 필터링**
- 행렬 분해(Matrix Factorization)를 이용하여 잠재 요인을 추출한다.
- 다차원 희소 행렬을 저차원 밀집 행렬로 바꾸어 사용한다.
- 행렬 분해에 의해 추출된 잠재요인을 특성 선호도로 가정할 수 있다.

행렬 분해는 다차원 -> 저차원 매트릭스로 변환하는 것으로 대표적으로 **SVD(Singular Vector Decomposition)**, **NMF(Non-Negative Matrix Factorization)** 등이 있다.

SVD는 **결측치가 없는 행렬에만 적용**을 할 수 있다. 결측치가 많은 경우에는 **SGD**, **ALS** 방식을 이용한다.

#### SGD를 활용한 행렬 분해

In [4]:
import numpy as np

In [5]:
R = np.array([[4, np.NaN, np.NaN, 2, np.NaN],
             [np.NaN, 5, np.NaN, 3, 1],
             [np.NaN, np.NaN, 3, 4, 4],
             [5, 2, 1, 2, np.NaN]])

User는 4명, Item은 5개

In [6]:
n_users, n_items = R.shape

In [7]:
K = 3 # 잠재 요인 차원

In [8]:
n_users, n_items

(4, 5)

In [9]:
np.random.seed(42)

In [10]:
P = np.random.normal(scale = 1. / K, size = (n_users, K))
Q = np.random.normal(scale = 1. / K, size = (n_items, K))

In [11]:
P.shape, Q.shape

((4, 3), (5, 3))

In [12]:
from sklearn.metrics import mean_squared_error

In [13]:
non_zeros = [(i, j, R[i, j]) for i in range(n_users) for j in range(n_items) if R[i, j] > 0]

In [14]:
non_zeros

[(0, 0, 4.0),
 (0, 3, 2.0),
 (1, 1, 5.0),
 (1, 3, 3.0),
 (1, 4, 1.0),
 (2, 2, 3.0),
 (2, 3, 4.0),
 (2, 4, 4.0),
 (3, 0, 5.0),
 (3, 1, 2.0),
 (3, 2, 1.0),
 (3, 3, 2.0)]

R 행렬에서 값이 0 보다 큰 위치의 행, 열 인덱스와 해당 값을 튜플 형태로 만들어 리스트에 저장

In [15]:
[v[0] for v in non_zeros]

[0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 3]

In [16]:
def get_rmse_err(R, P, Q, non_zeros) :
    err = 0
    
    matrix = np.dot(P, Q.T)
    
    row_idx = [v[0] for v in non_zeros]
    col_idx = [v[1] for v in non_zeros]
    
    new_R = R[row_idx, col_idx]
    
    non_zero_matrix = matrix[row_idx, col_idx]
    rmse = mean_squared_error(new_R, non_zero_matrix) ** 0.5
    return rmse

In [17]:
iterations = 1000
lr = 0.01
l2 = 0.01

In [18]:
for iteration in range(iterations) :
    for i, j, r in non_zeros :
        err_ij = r - np.dot(P[i, :], Q[j, :].T)
        
        P[i, :] = P[i, :] + lr * (err_ij * Q[j, :] - l2 * P[i, :])
        Q[j, :] = Q[j, :] + lr * (err_ij * P[i, :] - l2 * Q[j, :])
    
    rmse = get_rmse_err(R, P, Q, non_zeros)
    
    if (iteration % 100) == 0 :
        print(f'{iteration}번째 iteration step의 RMSE = {rmse}')
                                

0번째 iteration step의 RMSE = 3.2920697149118254
100번째 iteration step의 RMSE = 0.19350618778208667
200번째 iteration step의 RMSE = 0.030945925295917408
300번째 iteration step의 RMSE = 0.017061134025438924
400번째 iteration step의 RMSE = 0.015204135981955986
500번째 iteration step의 RMSE = 0.014871950471629453
600번째 iteration step의 RMSE = 0.014796657017388746
700번째 iteration step의 RMSE = 0.014758447296047143
800번째 iteration step의 RMSE = 0.014720787548844509
900번째 iteration step의 RMSE = 0.014680536323574913


In [19]:
pred_matrix = np.dot(P, Q.T)

In [20]:
pred_matrix

array([[3.98965296, 2.48787536, 1.34068206, 2.00231596, 1.1846934 ],
       [6.10970082, 4.97423562, 2.24094343, 2.99049244, 1.00743279],
       [5.40354633, 3.72248629, 2.98826537, 3.98054225, 3.98350837],
       [4.9725082 , 2.00940108, 1.0034686 , 1.99766709, 1.07702247]])

In [21]:
R

array([[ 4., nan, nan,  2., nan],
       [nan,  5., nan,  3.,  1.],
       [nan, nan,  3.,  4.,  4.],
       [ 5.,  2.,  1.,  2., nan]])

SGD를 활용해 진행한 행렬분해로 dot product한 결과와 기존 행렬인 R의 값이 거의 유사하다. 또한 결측치 값들이 예측값으로 채워진 것을 알 수 있다.